In [7]:
#Imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os

from env import get_connection

In [8]:
url = get_connection('curriculum_logs')

In [9]:
query = '''
SELECT *
FROM logs
'''

In [11]:
if os.path.isfile('curriculum_logs.csv'):
    
    df = pd.read_csv('curriculum_logs.csv')
    
else:
    url = get_connection('curriculum_logs')
    
    query = '''
            SELECT *
            FROM logs
            '''
    
    df = pd.read_sql(query, url)

In [13]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df.dtypes

Unnamed: 0             int64
date          datetime64[ns]
time                  object
path                  object
user_id                int64
cohort_id            float64
ip                    object
dtype: object

In [14]:
#Set as index
df = df.set_index('date')
df.head()

,Unnamed: 0,time,path,user_id,cohort_id,ip
date,,,,,,
2018-01-26,0,09:55:03,/,1,8.0,97.105.19.61
2018-01-26,1,09:56:02,java-ii,1,8.0,97.105.19.61
2018-01-26,2,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
2018-01-26,3,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
2018-01-26,4,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [15]:
df.drop(columns = 'Unnamed: 0', inplace = True)

In [16]:
df

,time,path,user_id,cohort_id,ip
date,,,,,
2018-01-26,09:55:03,/,1,8.0,97.105.19.61
2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61
...,...,...,...,...,...
2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33
2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33
2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33


In [17]:
daily_hits = df['path'].resample('d').count()
daily_hits.head()

date
2018-01-26    572
2018-01-27    230
2018-01-28    170
2018-01-29    830
2018-01-30    652
Freq: D, Name: path, dtype: int64

In [18]:
df['cohort_id'].value_counts().sort_index()

1.0       8890
2.0         93
4.0          4
5.0          1
6.0         72
7.0        598
8.0       1712
9.0          5
11.0       253
12.0       302
13.0      2845
14.0      9587
15.0       691
16.0       755
17.0      4954
18.0      2158
19.0      1237
21.0      7444
22.0     30926
23.0     28534
24.0     35636
25.0     25586
26.0     27749
27.0     20743
28.0     84031
29.0     38096
31.0     25359
32.0     29356
33.0     40730
34.0     26538
51.0     32888
52.0     28033
53.0     36902
55.0     21582
56.0     33568
57.0     33844
58.0     29855
59.0     32015
61.0     17713
62.0     37109
132.0    23691
133.0    14715
134.0    16623
135.0    16397
137.0     8562
138.0     7276
139.0     1672
Name: cohort_id, dtype: int64

In [21]:
df['user_id'].value_counts().sort_index()

1      7404
2      1541
3      1562
4       692
5      1701
       ... 
977      86
978      55
979     127
980       3
981      42
Name: user_id, Length: 981, dtype: int64

In [22]:
 def compute_bollinger(series, column, span, k):
    
    mean_df = series.ewm(span = span).mean()
    
    std_df =series.ewm(span = span).std()
    
    upper_band = mean_df + std_df * k
    
    lower_band = mean_df - std_df * k
    
    final_df = pd.concat([series, mean_df, upper_band, lower_band], axis = 1)
    
    final_df.columns = [column, 'midband', 'ub', 'lb']
    
    final_df['pct_b'] = (final_df[column] - final_df['lb'])/ (final_df['ub'] - final_df['lb'])
    
    return final_df

In [23]:
compute_bollinger(daily_hits, 'hits', 14 , 2)

,hits,midband,ub,lb,pct_b
date,,,,,
2018-01-26,572,572.000000,NaN,NaN,NaN
2018-01-27,230,388.785714,872.446753,-94.875324,0.335850
2018-01-28,170,305.208829,725.773959,-115.356302,0.339253
2018-01-29,830,465.757252,1116.800959,-185.286456,0.779738
2018-01-30,652,514.347700,1087.037023,-58.341623,0.620181
...,...,...,...,...,...
2021-04-17,478,1348.173718,2797.497736,-101.150300,0.199800
2021-04-18,659,1256.283889,2690.471058,-177.903279,0.291769
2021-04-19,2156,1376.246037,2854.618836,-102.126761,0.763720


In [24]:
compute_bollinger(daily_hits, 'hits', 30 , 2)

,hits,midband,ub,lb,pct_b
date,,,,,
2018-01-26,572,572.000000,NaN,NaN,NaN
2018-01-27,230,395.300000,878.961038,-88.361038,0.329116
2018-01-28,170,315.139578,742.895854,-112.616698,0.330348
2018-01-29,830,457.002386,1089.938060,-175.933288,0.794657
2018-01-30,652,501.369064,1070.328518,-67.590389,0.632374
...,...,...,...,...,...
2021-04-17,478,1325.179115,2805.204187,-154.845956,0.213796
2021-04-18,659,1282.199817,2751.889069,-187.489435,0.287982
2021-04-19,2156,1338.574023,2825.623313,-148.475267,0.774848
